In [1]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix

In [2]:
'''
# Cargo de Google Drive los archivos

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id_train_transaction='19setxTbNvfO5L5h-9h5-bcFcl7Ml1kHt'
downloaded = drive.CreateFile({'id': id_train_transaction})
downloaded.GetContentFile('train_transaction.csv')

id_train_identity = '1XUUs36t6K2ByA7xuQuTzRH_smXVtmVjF'
downloaded = drive.CreateFile({'id': id_train_identity})
downloaded.GetContentFile('train_identity.csv')

id_test_transaction='1LnLkTi7YYu95qlR6r7fDGbQUJrNR2uYE'
downloaded = drive.CreateFile({'id': id_test_transaction})
downloaded.GetContentFile('test_transaction.csv')

id_test_identity = '1Pro4HlgAnTo2UmGhS7NReITPAJxxPolA'
downloaded = drive.CreateFile({'id': id_test_identity})
downloaded.GetContentFile('test_identity.csv')'''

"\n# Cargo de Google Drive los archivos\n\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials\n\n\nauth.authenticate_user()\ngauth = GoogleAuth()\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\n\nid_train_transaction='19setxTbNvfO5L5h-9h5-bcFcl7Ml1kHt'\ndownloaded = drive.CreateFile({'id': id_train_transaction})\ndownloaded.GetContentFile('train_transaction.csv')\n\nid_train_identity = '1XUUs36t6K2ByA7xuQuTzRH_smXVtmVjF'\ndownloaded = drive.CreateFile({'id': id_train_identity})\ndownloaded.GetContentFile('train_identity.csv')\n\nid_test_transaction='1LnLkTi7YYu95qlR6r7fDGbQUJrNR2uYE'\ndownloaded = drive.CreateFile({'id': id_test_transaction})\ndownloaded.GetContentFile('test_transaction.csv')\n\nid_test_identity = '1Pro4HlgAnTo2UmGhS7NReITPAJxxPolA'\ndownloaded = drive.CreateFile({'id': id_test_identity})\ndownloaded.GetContentFile('tes

In [3]:
train_identity = pd.read_csv('./ieee-fraud-detection/train_identity.csv')

train_transaction = pd.read_csv('./ieee-fraud-detection/train_transaction.csv')

# Analizo los datasets

Creo los datasets encodeardos, limpios y escalados que voy a utilizar para entrenar y validar el modelo.

In [4]:
train_identity

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,-15.0,145955.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 66.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,F3111 Build/33.3.A.1.97
144229,3577526,-5.0,172059.0,NaN,NaN,1.0,-5.0,NaN,NaN,NaN,...,chrome 55.0 for android,32.0,855x480,match_status:2,T,F,T,F,mobile,A574BL Build/NMF26F
144230,3577529,-20.0,632381.0,NaN,NaN,-1.0,-36.0,NaN,NaN,NaN,...,chrome 65.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,Moto E (4) Plus Build/NMA26.42-152
144231,3577531,-5.0,55528.0,0.0,0.0,0.0,-7.0,NaN,NaN,0.0,...,chrome 66.0,24.0,2560x1600,match_status:2,T,F,T,F,desktop,MacOS


In [5]:
train_transaction

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> ***Observo que train_transaction tiene una columna de TransactionDT.***



Esta es una columna es de tipo datetime y representa el tiempo transcurrido desde un instante de referencia (desconocido). 

Esto significa que debe permanecer ordenada para evitar time-travel. 

In [6]:
# Verifico que esten ordenado los registros en el tiempo

list(train_transaction.TransactionDT) == sorted(train_transaction.TransactionDT, reverse=False)

True

## Busco el dataset completo con el que voy a trabajar

Uno los data sets de trainning que nos dan.

In [7]:
all_data = train_transaction.merge(train_identity, on='TransactionID', how='outer')

In [8]:
import gc

# Libero memoria del sistema
del(train_transaction)
del(train_identity)

# using the garbage collector to manually collect the garbage
gc.collect()

20

Este truco lo saque de Kaggle para administrar la memoria.

In [9]:
display(all_data)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Verifico que mantenga el orden segun TransactionDT

In [10]:
all_data.TransactionDT.hasnans

False

In [11]:
list(all_data.TransactionDT) == sorted(all_data.TransactionDT, reverse=False)

True

## Extraigo los target y features del dataset y creo los sets de trainning y testing.

Creo `data_train` y `data_test` para el set de `data` y

`label_train`, `label_test` para el set de `target`.

> Nota: El target es `isFraud`

### Particiono la data como train_set y test_set

Tomo un 20% de la data como `test_set` y el restante para `train_set` del modelo.

Me baso en TransactionDT como separador de registros.

In [12]:
labels = all_data.isFraud

In [13]:
# Tomo el 80% 
ochenta_porciento = int(all_data['TransactionDT'].size * 0.8)

In [14]:
data_train = all_data.iloc[: ochenta_porciento]

data_validation = all_data.iloc[ochenta_porciento:]

In [15]:
label_train = labels.iloc[:ochenta_porciento]

label_validation = labels.iloc[ochenta_porciento:]

## Limpio la data

Para los features categoricos reemplazo los 'nans' por 'desconocido' y para los features numericos los reemplazo por su mean de cada columna.

> Nota: Para el validation test reemplazo los nans por los means correspondientes a las columnas del trainning_set.
>
>  No se vuelven a calcular los means por columnas.

In [16]:
# Busco los promedios de cada columna numerica para el trainning_set

mean_values={}

for i in data_train.columns:
  if (str(data_train[i].dtype) != 'object'):
    mean_values[i] = data_train[i].mean()

In [17]:
# Limpio la data

def limpiar_dataset(dataset, mean_values):
  
  for i in dataset.columns:

    if (str(dataset[i].dtype) != 'object'):
      dataset[i].fillna(mean_values[i], inplace=True)
    
    else:
      dataset[i].fillna('desconocido',inplace=True)

In [18]:
limpiar_dataset(data_train, mean_values)
display(data_train)

/home/paredesj/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4535: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.50,W,13926,362.861214,150.0,discover,142.0,...,desconocido,26.596037,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido
1,2987001,0,86401,29.00,W,2755,404.000000,150.0,mastercard,102.0,...,desconocido,26.596037,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido
2,2987002,0,86469,59.00,W,4663,490.000000,150.0,visa,166.0,...,desconocido,26.596037,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido
3,2987003,0,86499,50.00,W,18132,567.000000,150.0,mastercard,117.0,...,desconocido,26.596037,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido
4,2987004,0,86506,50.00,H,4497,514.000000,150.0,mastercard,102.0,...,samsung browser 6.2,32.000000,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472427,3459427,0,12192667,43.95,W,15484,418.000000,150.0,visa,226.0,...,desconocido,26.596037,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido
472428,3459428,0,12192736,49.00,W,17188,321.000000,150.0,visa,226.0,...,desconocido,26.596037,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido
472429,3459429,0,12192742,40.00,H,16659,170.000000,150.0,visa,226.0,...,chrome 65.0,24.000000,1440x900,match_status:2,T,F,T,T,desktop,MacOS
472430,3459430,0,12192743,15.00,W,7919,194.000000,150.0,mastercard,166.0,...,desconocido,26.596037,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido


In [19]:
data_validation

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
472432,3459432,1,12192900,33.261,C,9300,103.0,185.0,visa,138.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,NaN
472433,3459433,0,12192911,52.811,C,8809,179.0,106.0,visa,137.0,...,chrome generic for android,NaN,NaN,NaN,F,F,T,F,mobile,F3311
472434,3459434,0,12192913,136.956,C,10819,555.0,185.0,visa,226.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
472435,3459435,0,12193040,136.956,C,9633,130.0,185.0,visa,138.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
472436,3459436,0,12193199,25.000,H,17188,321.0,150.0,visa,226.0,...,firefox 59.0,24.0,3200x1800,match_status:2,T,F,T,T,desktop,Windows
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.000,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.500,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.950,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.000,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
limpiar_dataset(data_validation, mean_values)
display(data_validation)

/home/paredesj/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4535: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
472432,3459432,1,12192900,33.261,C,9300,103.000000,185.0,visa,138.0,...,chrome 65.0,26.596037,desconocido,desconocido,F,F,T,F,desktop,desconocido
472433,3459433,0,12192911,52.811,C,8809,179.000000,106.0,visa,137.0,...,chrome generic for android,26.596037,desconocido,desconocido,F,F,T,F,mobile,F3311
472434,3459434,0,12192913,136.956,C,10819,555.000000,185.0,visa,226.0,...,chrome 65.0,26.596037,desconocido,desconocido,F,F,T,F,desktop,Windows
472435,3459435,0,12193040,136.956,C,9633,130.000000,185.0,visa,138.0,...,chrome 65.0,26.596037,desconocido,desconocido,F,F,T,F,desktop,Windows
472436,3459436,0,12193199,25.000,H,17188,321.000000,150.0,visa,226.0,...,firefox 59.0,24.000000,3200x1800,match_status:2,T,F,T,T,desktop,Windows
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.000,W,6550,362.861214,150.0,visa,226.0,...,desconocido,26.596037,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido
590536,3577536,0,15811049,39.500,W,10444,225.000000,150.0,mastercard,224.0,...,desconocido,26.596037,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido
590537,3577537,0,15811079,30.950,W,12037,595.000000,150.0,mastercard,224.0,...,desconocido,26.596037,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido
590538,3577538,0,15811088,117.000,W,7826,481.000000,150.0,mastercard,224.0,...,desconocido,26.596037,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido,desconocido


## Aplico Ordinal Encoding a data_train y luego a validation_train

Guardo los encodings para los features categoricos en un diccionario para luego asignarselos al set de test.

In [21]:
def obtener_columnas_categoricas(data_set):

  columnas_categoricas = []

  for col in data_set.columns:
        if (str(data_set[col].dtype) == 'object'):
            columnas_categoricas.append(col)

  return columnas_categoricas

In [22]:
# Creo el Ordinal Encoder por columna categoricas

from sklearn.preprocessing import OrdinalEncoder


oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
oe.fit(data_train[obtener_columnas_categoricas(data_train)])

OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

In [23]:
# Aplico el encoder al data_train
data_train[obtener_columnas_categoricas(data_train)] = oe.transform(data_train[obtener_columnas_categoricas(data_train)])
display(data_train)

/home/paredesj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/paredesj/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.50,4.0,13926,362.861214,150.0,2.0,142.0,...,60.0,26.596037,212.0,0.0,2.0,2.0,2.0,2.0,0.0,1578.0
1,2987001,0,86401,29.00,4.0,2755,404.000000,150.0,3.0,102.0,...,60.0,26.596037,212.0,0.0,2.0,2.0,2.0,2.0,0.0,1578.0
2,2987002,0,86469,59.00,4.0,4663,490.000000,150.0,4.0,166.0,...,60.0,26.596037,212.0,0.0,2.0,2.0,2.0,2.0,0.0,1578.0
3,2987003,0,86499,50.00,4.0,18132,567.000000,150.0,3.0,117.0,...,60.0,26.596037,212.0,0.0,2.0,2.0,2.0,2.0,0.0,1578.0
4,2987004,0,86506,50.00,1.0,4497,514.000000,150.0,3.0,102.0,...,104.0,32.000000,135.0,4.0,1.0,0.0,1.0,1.0,2.0,870.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472427,3459427,0,12192667,43.95,4.0,15484,418.000000,150.0,4.0,226.0,...,60.0,26.596037,212.0,0.0,2.0,2.0,2.0,2.0,0.0,1578.0
472428,3459428,0,12192736,49.00,4.0,17188,321.000000,150.0,4.0,226.0,...,60.0,26.596037,212.0,0.0,2.0,2.0,2.0,2.0,0.0,1578.0
472429,3459429,0,12192742,40.00,1.0,16659,170.000000,150.0,4.0,226.0,...,52.0,24.000000,62.0,4.0,1.0,0.0,1.0,1.0,1.0,661.0
472430,3459430,0,12192743,15.00,4.0,7919,194.000000,150.0,3.0,166.0,...,60.0,26.596037,212.0,0.0,2.0,2.0,2.0,2.0,0.0,1578.0


In [24]:
# Aplico el encoder al data_validation
data_validation[obtener_columnas_categoricas(data_validation)] = oe.transform(data_validation[obtener_columnas_categoricas(data_validation)])
display(data_validation)

/home/paredesj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/paredesj/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
472432,3459432,1,12192900,33.261,0.0,9300,103.000000,185.0,4.0,138.0,...,52.0,26.596037,212.0,0.0,0.0,0.0,1.0,0.0,1.0,1578.0
472433,3459433,0,12192911,52.811,0.0,8809,179.000000,106.0,4.0,137.0,...,56.0,26.596037,212.0,0.0,0.0,0.0,1.0,0.0,2.0,250.0
472434,3459434,0,12192913,136.956,0.0,10819,555.000000,185.0,4.0,226.0,...,52.0,26.596037,212.0,0.0,0.0,0.0,1.0,0.0,1.0,1467.0
472435,3459435,0,12193040,136.956,0.0,9633,130.000000,185.0,4.0,138.0,...,52.0,26.596037,212.0,0.0,0.0,0.0,1.0,0.0,1.0,1467.0
472436,3459436,0,12193199,25.000,1.0,17188,321.000000,150.0,4.0,226.0,...,74.0,24.000000,172.0,4.0,1.0,0.0,1.0,1.0,1.0,1467.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.000,4.0,6550,362.861214,150.0,4.0,226.0,...,60.0,26.596037,212.0,0.0,2.0,2.0,2.0,2.0,0.0,1578.0
590536,3577536,0,15811049,39.500,4.0,10444,225.000000,150.0,3.0,224.0,...,60.0,26.596037,212.0,0.0,2.0,2.0,2.0,2.0,0.0,1578.0
590537,3577537,0,15811079,30.950,4.0,12037,595.000000,150.0,3.0,224.0,...,60.0,26.596037,212.0,0.0,2.0,2.0,2.0,2.0,0.0,1578.0
590538,3577538,0,15811088,117.000,4.0,7826,481.000000,150.0,3.0,224.0,...,60.0,26.596037,212.0,0.0,2.0,2.0,2.0,2.0,0.0,1578.0


## Arreglo la data para poder usar el modelo correctamente

In [25]:
data_train = data_train.drop(columns=['isFraud', 'TransactionID'])

In [26]:
data_validation = data_validation.drop(columns=['isFraud', 'TransactionID'])

In [27]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(data_train)

data_train = scaler.fit_transform(data_train)
data_validation = scaler.transform(data_validation)

In [28]:
data_train

array([[-1.58139727, -0.27741211,  0.56643981, ...,  0.53733463,
        -0.53376046,  0.29682117],
       [-1.58139699, -0.44368749,  0.56643981, ...,  0.53733463,
        -0.53376046,  0.29682117],
       [-1.58137788, -0.31740239,  0.56643981, ...,  0.53733463,
        -0.53376046,  0.29682117],
       ...,
       [ 1.81938916, -0.39738295, -1.50669426, ..., -0.91143876,
         1.00636035, -3.85828064],
       [ 1.81938944, -0.50262053,  0.56643981, ...,  0.53733463,
        -0.53376046,  0.29682117],
       [ 1.81941725, -0.35949742,  0.56643981, ...,  0.53733463,
        -0.53376046,  0.29682117]])

# Creo el Modelo de Random Forest y lo entreno

Teniendo la data ordenada, creo el modelo de Random Forest y lo entreno con `data_train` y `label_train`

In [29]:
model = RandomForestRegressor(random_state=1, n_jobs=-1, n_estimators=5,max_features=10, max_depth=10)

In [30]:
model.fit(data_train, label_train)

RandomForestRegressor(max_depth=10, max_features=10, n_estimators=5, n_jobs=-1,
                      random_state=1)

# Metricas

## Aucroc Score

In [31]:
roc_auc_score(label_train, model.predict(data_train))

0.858587655990664

In [32]:
roc_auc_score(label_validation, model.predict(data_validation))

0.8419019524461064

# Busqueda de Hiper-parametros

Busco hiper-parametros para `RandomForest` utilizando `GridSearchCV`

In [33]:
from scipy.stats import uniform
from sklearn.model_selection import GridSearchCV

Hiper Parametros de RandomForest

*   **n_estimators**: cantidad de árboles a construir (100)
*   **max_depth**: máxima profundidad de cada árbol
*   **min_samples_split**: la cantidad mínima de datos requeridos para splitear un nodo interno (2)
*   **min_samples_leaf**: cantidad mínima de datos requeridos para ser una hoja (1)
*   **max_features**: la cantidad de features a considerar cuando se busca el mejor split (n)

In [44]:
hiper_parametros = dict(n_estimators=[3,5,8],max_depth=[2,4,6,10],max_features=[4,6,10])

In [45]:
gridSearch = GridSearchCV(model, hiper_parametros, cv=5, n_jobs=-1, scoring='roc_auc')

In [46]:
search = gridSearch.fit(data_train, label_train)

## Busco los mejores coeficientes

In [47]:
search.best_params_

{'max_depth': 10, 'max_features': 6, 'n_estimators': 8}

In [48]:
search.best_estimator_

RandomForestRegressor(max_depth=10, max_features=6, n_estimators=8, n_jobs=-1,
                      random_state=1)

# Random Fores con los mejores hiper-parametros obtenidos 

Usando RandomSearch obtengo como Hiper-Parametros obtenidos



| Hiper-Parametro   |  Mejor Valor |  
|:---:|:---:|
|  C | 1.5  |  
|  Penalty |  'l2'  | 

Vuelvo a plantear RandomForest para ver como mejora el modelo.

In [53]:
model = RandomForestRegressor(random_state=1,max_depth=10, max_features=6, n_estimators=8, n_jobs=-1)

model.fit(data_train, label_train)

RandomForestRegressor(max_depth=10, max_features=6, n_estimators=8, n_jobs=-1,
                      random_state=1)

## Guardo el modelo final

In [54]:
import joblib

joblib.dump(model, "random_forest_model.pkl")

['random_forest_model.pkl']

In [55]:
roc_auc_score(label_train, model.predict(data_train))

0.8624054016190821

In [56]:
roc_auc_score(label_validation, model.predict(data_validation))

0.8410961384361388